In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../')

from ase.db import connect
import pyro
from src.models.bnn import BNN, NN
from src.models.nets.network import NetAtom
import torch
import pytorch_lightning as L
import torch.utils.data as data
import numpy as np
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from optuna import Study
import optuna
from optuna.trial import Trial

from src.datamodule.aenet_datamodule import AenetDataModule

/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
datamodele = AenetDataModule(data_dir='../data/H2O/train.in', batch_size=200, test_split=0.1, valid_split=0.1)
net = NetAtom(datamodele.input_size, datamodele.hidden_size, datamodele.species, datamodele.active_names, datamodele.alpha,'cpu')

/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 0 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/torch/utils/data/dataset.py:449: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [5]:
from torch.optim import Adam

model = NN(net, Adam)

In [17]:
trainer = L.Trainer(max_epochs = 100)
trainer.fit(model=model, datamodule=datamodele)
print(trainer.callback_metrics)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type    | Params
---------------------------------
0 | net  | NetAtom | 2.1 K 
---------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.008     Total estimated model params size (MB)


Epoch 84:  31%|███       | 20/65 [00:00<00:00, 86.73it/s, v_num=5]         {'mse/val': tensor(0.1579), 'mse/train': tensor(0.1393)}


/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [18]:
trainer.test(model=model, datamodule=datamodele, ckpt_path=trainer.checkpoint_callback.best_model_path)

Restoring states from the checkpoint path at /home/riccardo/bin/repos/aenet-bnn/notebooks/lightning_logs/version_5/checkpoints/epoch=83-step=5460.ckpt
Loaded model weights from the checkpoint at /home/riccardo/bin/repos/aenet-bnn/notebooks/lightning_logs/version_5/checkpoints/epoch=83-step=5460.ckpt


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 43.78it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         mse/test          │    0.16316314041614532    │
└───────────────────────────┴───────────────────────────┘

[{'mse/test': 0.16316314041614532}]

In [32]:
batch = datamodele.test_dataloader()[0]

l2_E = torch.zeros(1, device=net.device)
E_error = 0.0
N_data2_E = 0
N_data_energy = 0

grp_descrp, grp_energy, logic_reduce, grp_N_atom = batch[10], batch[11], batch[12], batch[14]
l2_E, N_data2_E = model.net.get_loss_RMSE( grp_descrp, grp_energy, logic_reduce, grp_N_atom)

N_data_energy += N_data2_E
N_data_E_tot = N_data2_E
loss_tot = (1.0-net.alpha)*torch.sqrt((l2_E)/N_data_E_tot)
E_error += l2_E.item()
E_error = np.sqrt(E_error/N_data_energy)
train_error = (1.0-net.alpha.item())*E_error

print(E_error, train_error, l2_E.item())

0.039508763455474494 0.039508763455474494 0.17950837482477322


In [33]:
grp_N_atom

tensor([192, 240, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 240, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 240, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 192,
        192, 192, 192, 192, 192, 192, 192, 192, 192, 192, 240, 240, 192, 192,
        192, 192, 192])

In [52]:
differences = (model.net.forward(grp_descrp, logic_reduce) - grp_energy)
l1 = torch.sum( (differences/grp_N_atom)**2 )
l1

tensor(0.1795, dtype=torch.float64, grad_fn=<SumBackward0>)

In [44]:
running_mae, running_mae = 0,0
error = torch.abs(model.net.forward(grp_descrp, logic_reduce) - grp_energy).sum().data
squared_error = ((model.net.forward(grp_descrp, logic_reduce) - grp_energy)*(model.net.forward(grp_descrp, logic_reduce) - grp_energy)).sum().data
running_mae += error
running_mse += squared_error

rmse = math.sqrt(running_mse/len(loader_test))
mae = running_mae/len(loader_test)

(tensor(690.6847, dtype=torch.float64), tensor(6738.9732, dtype=torch.float64))

In [47]:
import torch.nn.functional as F


pred = model.net.forward(grp_descrp, logic_reduce) 
target = grp_energy
F.(pred, target)

tensor(58.5998, dtype=torch.float64, grad_fn=<MseLossBackward0>)